In [1]:
import pandas as pd
import json
from sodapy import Socrata
import os
import geopandas as gpd
import datetime as dt
import matplotlib.pyplot as plt
from config2 import app_token
from geojson import Feature, FeatureCollection, Point

In [2]:
# ## PANDAS TO GEOJSON FUNCTION
# def df_to_geojson(df, properties, lat='latitude', lon='longitude'):
#     geojson = {'type':'FeatureCollection', 'features':[]}
#     for _, row in df.iterrows():
#         feature = {'type':'Feature',
#                    'properties':{},
#                    'geometry':{'type':'Point',
#                                'coordinates':[]}}
#         feature['geometry']['coordinates'] = [row[lon],row[lat]]
#         for prop in properties:
#             feature['properties'][prop] = row[prop]
#         geojson['features'].append(feature)
#     return geojson

In [3]:
# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
# client = Socrata("data.cityofchicago.org", none)

# Example authenticated client (needed for non-public datasets):
# client = Socrata("data.cityofchicago.org",
#                  "du5ugsgl0w1g4ql37x78pjbc5",
#                  username= "dr.harmony",
#                  password= "F3ar&Loathing")

# # Example authenticated client (needed for non-public datasets):
client = Socrata("data.cityofchicago.org",
                 app_token)
#                  username= username,
#                  password= password)

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("85ca-t3if", limit = 500000)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

In [44]:
results_df

,crash_record_id,crash_date,posted_speed_limit,traffic_control_device,device_condition,weather_condition,lighting_condition,first_crash_type,trafficway_type,alignment,...,crash_date_est_i,photos_taken_i,statements_taken_i,private_property_i,work_zone_i,work_zone_type,workers_present_i,dooring_i,rd_no,lane_cnt
0,3fa1443347f8f5acfbea4d70170b000cdcf7c7cd919a7f...,2020-10-13T00:12:00.000,30,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",PARKED MOTOR VEHICLE,DIVIDED - W/MEDIAN BARRIER,STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,fa733ebf92caa9413d5a8ed9c1dfc28f9a96869b7782b4...,2020-10-12T23:30:00.000,30,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",PARKED MOTOR VEHICLE,NOT DIVIDED,STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,d16c88cf5ba23c72e560ce69b4324a9d7ce8776ce7d896...,2020-10-12T22:45:00.000,25,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",PARKED MOTOR VEHICLE,NOT DIVIDED,STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3c4b2c7fedbba09e8889d40dd01d2a20144ad8c63c4444...,2020-10-12T22:30:00.000,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",REAR END,NOT DIVIDED,STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,079e6cac8d7ad9d88e50988066ed8fd4671bc48d2618f3...,2020-10-12T22:20:00.000,30,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",REAR END,NOT DIVIDED,STRAIGHT AND LEVEL,...,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447083,964aaaeb569e364886cfbdf89ca73e09ca15cd916d87b2...,2014-06-25T19:00:00.000,30,NO CONTROLS,NO CONTROLS,RAIN,DAYLIGHT,SIDESWIPE SAME DIRECTION,NOT DIVIDED,STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,JC321679,NaN
447084,1d0232afecbdfd01968555aa956a688fd6f55a2bd1984f...,2014-02-24T19:45:00.000,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,RAIN,DAYLIGHT,TURNING,NOT DIVIDED,STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HZ164689,NaN
447085,957783a4787318f005a7dbc920e4c84cb9ac8aa7329a62...,2014-01-21T07:40:00.000,30,YIELD,NO CONTROLS,CLEAR,DAYLIGHT,ANGLE,DIVIDED - W/MEDIAN (NOT RAISED),STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HZ122950,NaN
447086,f62e27317feb174811cf4fefeb9fa1064fea6c0619a873...,2014-01-18T18:14:00.000,30,NO CONTROLS,NO CONTROLS,CLEAR,DUSK,PARKED MOTOR VEHICLE,DIVIDED - W/MEDIAN BARRIER,STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,JB442550,NaN


In [45]:
results_df.dtypes

crash_record_id                  object
crash_date                       object
posted_speed_limit               object
traffic_control_device           object
device_condition                 object
weather_condition                object
lighting_condition               object
first_crash_type                 object
trafficway_type                  object
alignment                        object
roadway_surface_cond             object
road_defect                      object
report_type                      object
crash_type                       object
damage                           object
date_police_notified             object
prim_contributory_cause          object
sec_contributory_cause           object
street_no                        object
street_direction                 object
street_name                      object
beat_of_occurrence               object
num_units                        object
most_severe_injury               object
injuries_total                   object


In [78]:
results_df = results_df.dropna(subset = ['beat_of_occurrence'])
results_df = results_df.dropna(subset = ['latitude'])
results_df = results_df.dropna(subset = ['longitude'])
results_df

,crash_record_id,crash_date,posted_speed_limit,traffic_control_device,device_condition,weather_condition,lighting_condition,first_crash_type,trafficway_type,alignment,...,crash_date_est_i,photos_taken_i,statements_taken_i,private_property_i,work_zone_i,work_zone_type,workers_present_i,dooring_i,rd_no,lane_cnt
0,3fa1443347f8f5acfbea4d70170b000cdcf7c7cd919a7f...,2020-10-13 00:12:00,30,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",PARKED MOTOR VEHICLE,DIVIDED - W/MEDIAN BARRIER,STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,fa733ebf92caa9413d5a8ed9c1dfc28f9a96869b7782b4...,2020-10-12 23:30:00,30,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",PARKED MOTOR VEHICLE,NOT DIVIDED,STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,d16c88cf5ba23c72e560ce69b4324a9d7ce8776ce7d896...,2020-10-12 22:45:00,25,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",PARKED MOTOR VEHICLE,NOT DIVIDED,STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3c4b2c7fedbba09e8889d40dd01d2a20144ad8c63c4444...,2020-10-12 22:30:00,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",REAR END,NOT DIVIDED,STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,079e6cac8d7ad9d88e50988066ed8fd4671bc48d2618f3...,2020-10-12 22:20:00,30,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",REAR END,NOT DIVIDED,STRAIGHT AND LEVEL,...,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447083,964aaaeb569e364886cfbdf89ca73e09ca15cd916d87b2...,2014-06-25 19:00:00,30,NO CONTROLS,NO CONTROLS,RAIN,DAYLIGHT,SIDESWIPE SAME DIRECTION,NOT DIVIDED,STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,JC321679,NaN
447084,1d0232afecbdfd01968555aa956a688fd6f55a2bd1984f...,2014-02-24 19:45:00,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,RAIN,DAYLIGHT,TURNING,NOT DIVIDED,STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HZ164689,NaN
447085,957783a4787318f005a7dbc920e4c84cb9ac8aa7329a62...,2014-01-21 07:40:00,30,YIELD,NO CONTROLS,CLEAR,DAYLIGHT,ANGLE,DIVIDED - W/MEDIAN (NOT RAISED),STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HZ122950,NaN
447086,f62e27317feb174811cf4fefeb9fa1064fea6c0619a873...,2014-01-18 18:14:00,30,NO CONTROLS,NO CONTROLS,CLEAR,DUSK,PARKED MOTOR VEHICLE,DIVIDED - W/MEDIAN BARRIER,STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,JB442550,NaN


In [79]:
results_df['crash_date'] = pd.to_datetime(results_df['crash_date'])

In [80]:
results_df2019 = results_df[results_df['crash_date'].dt.year == 2019]
results_df2019

,crash_record_id,crash_date,posted_speed_limit,traffic_control_device,device_condition,weather_condition,lighting_condition,first_crash_type,trafficway_type,alignment,...,crash_date_est_i,photos_taken_i,statements_taken_i,private_property_i,work_zone_i,work_zone_type,workers_present_i,dooring_i,rd_no,lane_cnt
72478,032ca7e09951de4385eec78a50a5f9f33b217838578b29...,2019-12-31 23:58:00,35,NO CONTROLS,NO CONTROLS,SNOW,"DARKNESS, LIGHTED ROAD",PARKED MOTOR VEHICLE,NOT DIVIDED,STRAIGHT ON GRADE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,JD100044,NaN
72479,71065ad347ee197080534e112b27c8a5b6fb8d9bbde2c3...,2019-12-31 23:57:00,30,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",FIXED OBJECT,DIVIDED - W/MEDIAN BARRIER,STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,JD100010,NaN
72480,9c2724305c03bf461d72cbcf2879c96b86ed5e180b62be...,2019-12-31 23:50:00,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",FIXED OBJECT,DIVIDED - W/MEDIAN BARRIER,STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,JD100008,NaN
72481,3187e5839fab7ff654383ba44a2e5b430c856b3fa10dab...,2019-12-31 23:46:00,45,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",REAR END,DIVIDED - W/MEDIAN BARRIER,STRAIGHT AND LEVEL,...,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,JD100006,NaN
72482,449bc7511dba6e6d605411276dc08dbeeccff39b074db0...,2019-12-31 23:45:00,25,STOP SIGN/FLASHER,UNKNOWN,CLEAR,"DARKNESS, LIGHTED ROAD",PARKED MOTOR VEHICLE,NOT DIVIDED,STRAIGHT AND LEVEL,...,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,JD100842,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190221,cb22668bf69d9c99e50c615a9ff996328190859a0e8351...,2019-01-01 00:07:00,30,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",PARKED MOTOR VEHICLE,ONE-WAY,STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,JC100171,1
190222,caa1629056b16f2e6ec6797d6252d6da8ec1142c0ee982...,2019-01-01 00:04:00,20,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",TURNING,ONE-WAY,STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,JC100033,2
190223,0777b210b7a60ac1338d90e20520d421c3241971049303...,2019-01-01 00:01:00,30,NO CONTROLS,NO CONTROLS,UNKNOWN,"DARKNESS, LIGHTED ROAD",PARKED MOTOR VEHICLE,DIVIDED - W/MEDIAN (NOT RAISED),STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,JC100780,NaN
190224,cb2b9a910babe20dfad7fc27c04c29e7b199fa94dc57bf...,2019-01-01 00:00:00,35,NO CONTROLS,NO CONTROLS,UNKNOWN,UNKNOWN,PARKED MOTOR VEHICLE,NOT DIVIDED,STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,JC111663,2


In [81]:
crashes = results_df2019.drop(columns=['crash_record_id', 'location', 'injuries_unknown', 'injuries_incapacitating', 'injuries_no_indication', 'injuries_reported_not_evident', 'injuries_non_incapacitating', 'private_property_i', 'crash_date_est_i', 'statements_taken_i', 'photos_taken_i', 'work_zone_i', 'work_zone_type', 'workers_present_i', 'dooring_i'])

In [82]:
crashes.rename(columns={"beat_of_occurrence": "Beat"}, inplace = True)

In [83]:
crashes.dtypes

crash_date                 datetime64[ns]
posted_speed_limit                 object
traffic_control_device             object
device_condition                   object
weather_condition                  object
lighting_condition                 object
first_crash_type                   object
trafficway_type                    object
alignment                          object
roadway_surface_cond               object
road_defect                        object
report_type                        object
crash_type                         object
damage                             object
date_police_notified               object
prim_contributory_cause            object
sec_contributory_cause             object
street_no                          object
street_direction                   object
street_name                        object
Beat                               object
num_units                          object
most_severe_injury                 object
injuries_total                    

In [85]:
crashes.Beat.unique

<bound method Series.unique of 72478     1135
72479      814
72480     1512
72481     1824
72482     1223
          ... 
190221    1624
190222     122
190223    1922
190224     235
190225    1123
Name: Beat, Length: 116829, dtype: object>

In [86]:

crashes['Beat'] = pd.to_numeric(crashes['Beat'])
crashes.dtypes

crash_date                 datetime64[ns]
posted_speed_limit                 object
traffic_control_device             object
device_condition                   object
weather_condition                  object
lighting_condition                 object
first_crash_type                   object
trafficway_type                    object
alignment                          object
roadway_surface_cond               object
road_defect                        object
report_type                        object
crash_type                         object
damage                             object
date_police_notified               object
prim_contributory_cause            object
sec_contributory_cause             object
street_no                          object
street_direction                   object
street_name                        object
Beat                                int64
num_units                          object
most_severe_injury                 object
injuries_total                    

In [53]:
# Reference the file where the CSV is located
neighborhood_path = "static/data/ChicagoAreasBeatDistricts.csv"

# Import the data into a Pandas DataFrame
neighborhoods_df = pd.read_csv(neighborhood_path)
neighborhoods_df.dtypes

Beat              int64
District          int64
Community Area    int64
dtype: object

In [54]:
# crash_neighborhood_merge = pd.merge(left=crashes, right=neighborhoods_df, how='right', on=['Beat'])
# crash_neighborhood_merge = pd.merge(crashes, neighborhoods_df, on='Beat', how='right')

In [55]:
neighborhoods_df.shape

(506, 3)

In [75]:
crash_neighborhood_merge = neighborhoods_df.join(crashes, how='right', on='Beat', lsuffix='_left', rsuffix='_right')

In [76]:
crash_neighborhood_merge = crash_neighborhood_merge.drop_duplicates()
crash_neighborhood_merge

,Beat,Beat_left,District,Community Area,crash_date,posted_speed_limit,traffic_control_device,device_condition,weather_condition,lighting_condition,...,injuries_fatal,crash_hour,crash_day_of_week,crash_month,latitude,longitude,hit_and_run_i,intersection_related_i,rd_no,lane_cnt
NaN,72478,NaN,NaN,NaN,2019-12-31 23:58:00,35,NO CONTROLS,NO CONTROLS,SNOW,"DARKNESS, LIGHTED ROAD",...,0,23,3,12,41.871797864,-87.696189628,Y,NaN,JD100044,NaN
NaN,72479,NaN,NaN,NaN,2019-12-31 23:57:00,30,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",...,0,23,3,12,41.812437437,-87.743434278,NaN,Y,JD100010,NaN
NaN,72480,NaN,NaN,NaN,2019-12-31 23:50:00,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",...,NaN,23,3,12,41.887804621,-87.765268498,Y,NaN,JD100008,NaN
NaN,72481,NaN,NaN,NaN,2019-12-31 23:46:00,45,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",...,0,23,3,12,41.910943878,-87.626392797,NaN,NaN,JD100006,NaN
NaN,72482,NaN,NaN,NaN,2019-12-31 23:45:00,25,STOP SIGN/FLASHER,UNKNOWN,CLEAR,"DARKNESS, LIGHTED ROAD",...,0,23,3,12,41.883961501,-87.674114078,Y,Y,JD100842,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NaN,190221,NaN,NaN,NaN,2019-01-01 00:07:00,30,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",...,0,0,3,1,41.954529286,-87.784311465,Y,NaN,JC100171,1
NaN,190222,NaN,NaN,NaN,2019-01-01 00:04:00,20,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",...,0,0,3,1,41.882058105,-87.63347836,NaN,Y,JC100033,2
NaN,190223,NaN,NaN,NaN,2019-01-01 00:01:00,30,NO CONTROLS,NO CONTROLS,UNKNOWN,"DARKNESS, LIGHTED ROAD",...,0,0,3,1,41.94685643,-87.670809132,Y,NaN,JC100780,NaN
NaN,190224,NaN,NaN,NaN,2019-01-01 00:00:00,35,NO CONTROLS,NO CONTROLS,UNKNOWN,UNKNOWN,...,0,0,3,1,41.793502271,-87.586406738,Y,NaN,JC111663,2


In [21]:
# crash_neighborhood_merge = pd.concat([crashes, neighborhoods_df], axis=1)
crash_neighborhood_merge_clean = crash_neighborhood_merge[['crash_date', 'posted_speed_limit', 'traffic_control_device',
       'device_condition', 'weather_condition', 'lighting_condition',
       'first_crash_type', 'trafficway_type', 'alignment', 'injuries_fatal',
       'roadway_surface_cond', 'road_defect', 'report_type', 'crash_type',
       'intersection_related_i', 'damage', 'date_police_notified',
       'prim_contributory_cause', 'sec_contributory_cause', 'street_no',
       'street_direction', 'street_name', 'District', 'Community Area', 'Beat', 'num_units',
       'most_severe_injury', 'injuries_total', 'crash_hour',
       'crash_day_of_week', 'crash_month', 'latitude', 'longitude',
       'hit_and_run_i', 'rd_no', 'lane_cnt']]
crash_neighborhood_merge_clean

,crash_date,posted_speed_limit,traffic_control_device,device_condition,weather_condition,lighting_condition,first_crash_type,trafficway_type,alignment,injuries_fatal,...,most_severe_injury,injuries_total,crash_hour,crash_day_of_week,crash_month,latitude,longitude,hit_and_run_i,rd_no,lane_cnt
0,2020-10-13T00:12:00.000,30,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",PARKED MOTOR VEHICLE,DIVIDED - W/MEDIAN BARRIER,STRAIGHT AND LEVEL,0,...,NONINCAPACITATING INJURY,1,0,3,10,41.95361776,-87.836539774,NaN,NaN,NaN
1,2020-10-13T00:12:00.000,30,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",PARKED MOTOR VEHICLE,DIVIDED - W/MEDIAN BARRIER,STRAIGHT AND LEVEL,0,...,NONINCAPACITATING INJURY,1,0,3,10,41.95361776,-87.836539774,NaN,NaN,NaN
2,2020-10-13T00:12:00.000,30,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",PARKED MOTOR VEHICLE,DIVIDED - W/MEDIAN BARRIER,STRAIGHT AND LEVEL,0,...,NONINCAPACITATING INJURY,1,0,3,10,41.95361776,-87.836539774,NaN,NaN,NaN
3,2020-10-12T23:30:00.000,30,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",PARKED MOTOR VEHICLE,NOT DIVIDED,STRAIGHT AND LEVEL,0,...,NO INDICATION OF INJURY,0,23,2,10,41.780133017,-87.581620874,Y,NaN,NaN
4,2020-10-12T23:30:00.000,30,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",PARKED MOTOR VEHICLE,NOT DIVIDED,STRAIGHT AND LEVEL,0,...,NO INDICATION OF INJURY,0,23,2,10,41.780133017,-87.581620874,Y,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
866464,2014-01-21T07:40:00.000,30,YIELD,NO CONTROLS,CLEAR,DAYLIGHT,ANGLE,DIVIDED - W/MEDIAN (NOT RAISED),STRAIGHT AND LEVEL,0,...,NONINCAPACITATING INJURY,1,7,3,1,41.760710194,-87.56194603,NaN,HZ122950,NaN
866465,2014-01-21T07:40:00.000,30,YIELD,NO CONTROLS,CLEAR,DAYLIGHT,ANGLE,DIVIDED - W/MEDIAN (NOT RAISED),STRAIGHT AND LEVEL,0,...,NONINCAPACITATING INJURY,1,7,3,1,41.760710194,-87.56194603,NaN,HZ122950,NaN
866466,2014-01-18T18:14:00.000,30,NO CONTROLS,NO CONTROLS,CLEAR,DUSK,PARKED MOTOR VEHICLE,DIVIDED - W/MEDIAN BARRIER,STRAIGHT AND LEVEL,0,...,NO INDICATION OF INJURY,0,18,7,1,41.885609917,-87.63875619,NaN,JB442550,NaN
866467,2013-03-03T16:48:00.000,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,SIDESWIPE OPPOSITE DIRECTION,NOT DIVIDED,STRAIGHT AND LEVEL,0,...,"REPORTED, NOT EVIDENT",2,16,1,3,41.975856916,-87.708743642,Y,JC172482,2


In [22]:
crash_neighborhood_merge_clean.columns

Index(['crash_date', 'posted_speed_limit', 'traffic_control_device',
       'device_condition', 'weather_condition', 'lighting_condition',
       'first_crash_type', 'trafficway_type', 'alignment', 'injuries_fatal',
       'roadway_surface_cond', 'road_defect', 'report_type', 'crash_type',
       'intersection_related_i', 'damage', 'date_police_notified',
       'prim_contributory_cause', 'sec_contributory_cause', 'street_no',
       'street_direction', 'street_name', 'District', 'Community Area', 'Beat',
       'num_units', 'most_severe_injury', 'injuries_total', 'crash_hour',
       'crash_day_of_week', 'crash_month', 'latitude', 'longitude',
       'hit_and_run_i', 'rd_no', 'lane_cnt'],
      dtype='object')

In [23]:
crash_neighborhood_merge_clean

,crash_date,posted_speed_limit,traffic_control_device,device_condition,weather_condition,lighting_condition,first_crash_type,trafficway_type,alignment,injuries_fatal,...,most_severe_injury,injuries_total,crash_hour,crash_day_of_week,crash_month,latitude,longitude,hit_and_run_i,rd_no,lane_cnt
0,2020-10-13T00:12:00.000,30,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",PARKED MOTOR VEHICLE,DIVIDED - W/MEDIAN BARRIER,STRAIGHT AND LEVEL,0,...,NONINCAPACITATING INJURY,1,0,3,10,41.95361776,-87.836539774,NaN,NaN,NaN
1,2020-10-13T00:12:00.000,30,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",PARKED MOTOR VEHICLE,DIVIDED - W/MEDIAN BARRIER,STRAIGHT AND LEVEL,0,...,NONINCAPACITATING INJURY,1,0,3,10,41.95361776,-87.836539774,NaN,NaN,NaN
2,2020-10-13T00:12:00.000,30,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",PARKED MOTOR VEHICLE,DIVIDED - W/MEDIAN BARRIER,STRAIGHT AND LEVEL,0,...,NONINCAPACITATING INJURY,1,0,3,10,41.95361776,-87.836539774,NaN,NaN,NaN
3,2020-10-12T23:30:00.000,30,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",PARKED MOTOR VEHICLE,NOT DIVIDED,STRAIGHT AND LEVEL,0,...,NO INDICATION OF INJURY,0,23,2,10,41.780133017,-87.581620874,Y,NaN,NaN
4,2020-10-12T23:30:00.000,30,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",PARKED MOTOR VEHICLE,NOT DIVIDED,STRAIGHT AND LEVEL,0,...,NO INDICATION OF INJURY,0,23,2,10,41.780133017,-87.581620874,Y,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
866464,2014-01-21T07:40:00.000,30,YIELD,NO CONTROLS,CLEAR,DAYLIGHT,ANGLE,DIVIDED - W/MEDIAN (NOT RAISED),STRAIGHT AND LEVEL,0,...,NONINCAPACITATING INJURY,1,7,3,1,41.760710194,-87.56194603,NaN,HZ122950,NaN
866465,2014-01-21T07:40:00.000,30,YIELD,NO CONTROLS,CLEAR,DAYLIGHT,ANGLE,DIVIDED - W/MEDIAN (NOT RAISED),STRAIGHT AND LEVEL,0,...,NONINCAPACITATING INJURY,1,7,3,1,41.760710194,-87.56194603,NaN,HZ122950,NaN
866466,2014-01-18T18:14:00.000,30,NO CONTROLS,NO CONTROLS,CLEAR,DUSK,PARKED MOTOR VEHICLE,DIVIDED - W/MEDIAN BARRIER,STRAIGHT AND LEVEL,0,...,NO INDICATION OF INJURY,0,18,7,1,41.885609917,-87.63875619,NaN,JB442550,NaN
866467,2013-03-03T16:48:00.000,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,SIDESWIPE OPPOSITE DIRECTION,NOT DIVIDED,STRAIGHT AND LEVEL,0,...,"REPORTED, NOT EVIDENT",2,16,1,3,41.975856916,-87.708743642,Y,JC172482,2


In [24]:
## DROPPING COLUMNS
crashes_in_areas = crash_neighborhood_merge_clean.drop(columns=['posted_speed_limit', 'traffic_control_device', 'crash_type', 'device_condition', 'weather_condition', 'lighting_condition', 'trafficway_type', 'alignment',
'roadway_surface_cond', 'road_defect', 'report_type','intersection_related_i', 'damage', 'date_police_notified',
'prim_contributory_cause', 'sec_contributory_cause', 'street_no',
'street_direction', 'street_name', 'District', 'num_units', 'most_severe_injury', 'crash_hour',
'crash_day_of_week', 'crash_month', 'rd_no', 'lane_cnt', 'hit_and_run_i'])

In [25]:
crashes_in_areas

,crash_date,first_crash_type,injuries_fatal,Community Area,Beat,injuries_total,latitude,longitude
0,2020-10-13T00:12:00.000,PARKED MOTOR VEHICLE,0,10.0,1614.0,1,41.95361776,-87.836539774
1,2020-10-13T00:12:00.000,PARKED MOTOR VEHICLE,0,17.0,1614.0,1,41.95361776,-87.836539774
2,2020-10-13T00:12:00.000,PARKED MOTOR VEHICLE,0,76.0,1614.0,1,41.95361776,-87.836539774
3,2020-10-12T23:30:00.000,PARKED MOTOR VEHICLE,0,41.0,331.0,0,41.780133017,-87.581620874
4,2020-10-12T23:30:00.000,PARKED MOTOR VEHICLE,0,42.0,331.0,0,41.780133017,-87.581620874
...,...,...,...,...,...,...,...,...
866464,2014-01-21T07:40:00.000,ANGLE,0,43.0,334.0,1,41.760710194,-87.56194603
866465,2014-01-21T07:40:00.000,ANGLE,0,43.0,334.0,1,41.760710194,-87.56194603
866466,2014-01-18T18:14:00.000,PARKED MOTOR VEHICLE,0,28.0,121.0,0,41.885609917,-87.63875619
866467,2013-03-03T16:48:00.000,SIDESWIPE OPPOSITE DIRECTION,0,13.0,1712.0,2,41.975856916,-87.708743642


In [26]:
## DROP any row with NaN values ** THIS REMOVES SOME CRASHES BUT KEEPS HIGH QUALITY RECORDED **
df = crashes_in_areas.dropna()
df

,crash_date,first_crash_type,injuries_fatal,Community Area,Beat,injuries_total,latitude,longitude
0,2020-10-13T00:12:00.000,PARKED MOTOR VEHICLE,0,10.0,1614.0,1,41.95361776,-87.836539774
1,2020-10-13T00:12:00.000,PARKED MOTOR VEHICLE,0,17.0,1614.0,1,41.95361776,-87.836539774
2,2020-10-13T00:12:00.000,PARKED MOTOR VEHICLE,0,76.0,1614.0,1,41.95361776,-87.836539774
3,2020-10-12T23:30:00.000,PARKED MOTOR VEHICLE,0,41.0,331.0,0,41.780133017,-87.581620874
4,2020-10-12T23:30:00.000,PARKED MOTOR VEHICLE,0,42.0,331.0,0,41.780133017,-87.581620874
...,...,...,...,...,...,...,...,...
866464,2014-01-21T07:40:00.000,ANGLE,0,43.0,334.0,1,41.760710194,-87.56194603
866465,2014-01-21T07:40:00.000,ANGLE,0,43.0,334.0,1,41.760710194,-87.56194603
866466,2014-01-18T18:14:00.000,PARKED MOTOR VEHICLE,0,28.0,121.0,0,41.885609917,-87.63875619
866467,2013-03-03T16:48:00.000,SIDESWIPE OPPOSITE DIRECTION,0,13.0,1712.0,2,41.975856916,-87.708743642


In [27]:
## rename Community Area and Beat columns names
df = df.rename(columns = {"Community Area": "communityArea", "Beat": "beat"})
df

,crash_date,first_crash_type,injuries_fatal,communityArea,beat,injuries_total,latitude,longitude
0,2020-10-13T00:12:00.000,PARKED MOTOR VEHICLE,0,10.0,1614.0,1,41.95361776,-87.836539774
1,2020-10-13T00:12:00.000,PARKED MOTOR VEHICLE,0,17.0,1614.0,1,41.95361776,-87.836539774
2,2020-10-13T00:12:00.000,PARKED MOTOR VEHICLE,0,76.0,1614.0,1,41.95361776,-87.836539774
3,2020-10-12T23:30:00.000,PARKED MOTOR VEHICLE,0,41.0,331.0,0,41.780133017,-87.581620874
4,2020-10-12T23:30:00.000,PARKED MOTOR VEHICLE,0,42.0,331.0,0,41.780133017,-87.581620874
...,...,...,...,...,...,...,...,...
866464,2014-01-21T07:40:00.000,ANGLE,0,43.0,334.0,1,41.760710194,-87.56194603
866465,2014-01-21T07:40:00.000,ANGLE,0,43.0,334.0,1,41.760710194,-87.56194603
866466,2014-01-18T18:14:00.000,PARKED MOTOR VEHICLE,0,28.0,121.0,0,41.885609917,-87.63875619
866467,2013-03-03T16:48:00.000,SIDESWIPE OPPOSITE DIRECTION,0,13.0,1712.0,2,41.975856916,-87.708743642


In [28]:
df.dtypes

crash_date           object
first_crash_type     object
injuries_fatal       object
communityArea       float64
beat                float64
injuries_total       object
latitude             object
longitude            object
dtype: object

In [31]:
## READ Chicago Area geoJSON file for merge
chicagoAreas_df = gpd.read_file("static/data/chicago-community-areas.geojson")

In [ ]:
chicagoAreas_df
print(chicagoAreas_df)

In [ ]:
## REMOVE ** "area", "perimeter", "area_num_1", "comarea_id", "comarea"   ** from chicagoAreas_df
chicagoAreas_df = chicagoAreas_df.drop(columns=["area", "perimeter", "area_num_1", "comarea_id", "comarea"])
chicagoAreas_df = chicagoAreas_df.rename(columns = {"area_numbe": "communityArea"})
chicagoAreas_df.dtypes

In [ ]:
chicagoAreas_df[["communityArea"]] = chicagoAreas_df[["communityArea"]].apply(pd.to_numeric)
chicagoAreas_df.dtypes

In [ ]:
## MERGE THE TWO FILES TO CREATE MASTER 2019 GEOJSON
chicagoCrashMerge = pd.merge(left=df2019, right=chicagoAreas_df, how='right', on=['communityArea'])
chicagoCrashMerge

In [ ]:
chicagoCrashMerge['crashes_per_area'] = chicagoCrashMerge.groupby(['communityArea'])['community'].transform('count')
chicagoCrashMerge

In [ ]:
# CREATING GEOJSON FILE FOR EXPORT

## USING THE GEOPANDAS
# gdf_crashes2019 = geopandas.GeoDataFrame(
#     df, geometry=geopandas.points_from_xy(df.longitude, df.latitude))
gdf_crashes2019 = gpd.GeoDataFrame(chicagoCrashMerge)

In [ ]:
print(gdf_crashes2019.head())

In [ ]:
ax = gdf_crashes2019.dropna().plot(column='crashes_per_area', cmap =    
                                'YlGnBu', figsize=(15,9),   
                                 scheme='quantiles', k=3, legend =  
                                  True);
#add title to the map
ax.set_title('Number of Crashes Per Chicago Area', fontdict= 
            {'fontsize':25})
#remove axes
ax.set_axis_off()
#move legend to an empty space
ax.get_legend().set_bbox_to_anchor((.12,.12))
ax.get_figure()

In [ ]:
#gdf_crashes2019.to_file("crashes_in_2019.geojson", driver='GeoJSON')